In [1]:
import nibabel as nib
import numpy as np
import pydicom
from tkinter import filedialog
import tkinter as tk
import glob
from tqdm import tqdm
import torch
import h5py
import matplotlib.pyplot as plt
import os
import time
import pandas as pd

In [2]:
def tkinter_gui(flavour_text):
    """
    This function creates a Tkinter window for browsing directories and returns the selected directory path.

    Parameters
    ----------
    flavour_text : str
        A string to display in the Tkinter window.

    Returns
    -------
    selected_directory : str
        The selected directory path.

    Examples
    --------
    >>> selected_directory = tkinter_gui("Select a directory")
    """
    
    def browse_directory():
        directory_path = filedialog.askdirectory()
        if directory_path:
            nonlocal selected_directory
            selected_directory = directory_path
            root.destroy()  # Close the Tkinter window

    # Create the main window
    root = tk.Tk()
    root.title("Directory Browser")

    # Create a label and button to display selected directory and browse
    directory_label = tk.Label(root, text=flavour_text, padx=10, pady=10)
    directory_label.pack()
    browse_button = tk.Button(root, text="Browse", command=browse_directory)
    browse_button.pack()

    # Run the application
    selected_directory = None  # Initialize selected_directory variable
    root.mainloop()
    if selected_directory:
        return selected_directory   
    else:
        raise ValueError("No directory selected")
    

def save_pipeline(image, orig_filepath, savepath, counter, dataset_name="denoised"):

    savepath = "C:\\Users\\miker\\Downloads\\test2"
    image = np.array(image)

    if ".dcm" in orig_filepath:
        mode = "dcm"
    elif ".nii" in orig_filepath:
        mode = "nii"
    elif ".h5" in orig_filepath:
        mode = "h5"

    if mode == "dcm":
        dcm = pydicom.dcmread(orig_filepath)
        
        assert dcm.pixel_array.shape == image.shape, "Image shape does not match the original image shape"
        dcm.PixelData = image.tobytes()
        #other parameters to change?
        dcm.save_as(savepath + "\\" + dataset_name + "_" + str(counter) + ".dcm")

    elif mode == "nii":
        nii = nib.load(orig_filepath)
        nii = nib.Nifti1Image(image, nii.affine)
        nib.save(nii, savepath + "\\" + dataset_name + "_" + str(counter) + ".nii")

    elif mode == "h5":
        with h5py.File('original_file.h5', 'a') as original_h5:
            assert original_h5['pixel_data'].shape == image.shape, "Image shape does not match the original image shape"
            with h5py.File('modified_file.h5', 'w') as modified_h5:
                # Copy the modified dataset to the new file
                modified_h5.create_dataset('pixel_data', data=image, dtype=np.float32)
                # Copy the other datasets to the new file
                for key in original_h5.keys():
                    if key != 'pixel_data':
                        original_h5.copy(key, modified_h5)
        os.rename('modified_file.h5', savepath + "\\" + dataset_name + "_" + str(counter) + ".h5")

    return

In [23]:
rootdir = tkinter_gui("Select the directory containing the network output denoised .npy file and the dataloc container file")

dataloc_container = glob.glob(os.path.join(rootdir + '/*.csv'))
assert len(dataloc_container) == 1, "There should be one .csv file in the directory. Check if the dataloc container file is present in the directory."
dataloc_container = dataloc_container[0]
denoised_imgs = glob.glob(os.path.join(rootdir + '/*.npy'))
assert len(denoised_imgs) == 1, "There should be one .npy file in the directory. Check if the denoised .npy file is present in the directory."
denoised_imgs = denoised_imgs[0]

In [40]:
df = pd.read_csv(dataloc_container, index_col=0, header=None)
denoised_imgs_npy = np.load(denoised_imgs)

In [55]:
#get value from the dataframe
counter = 0
temp_container = []
full_container = []
for index, row in df.iterrows():
    if counter == 0:
        last_row = row
        temp_container.append(denoised_imgs_npy[counter])
    if last_row == row:
        temp_container.append(denoised_imgs_npy[counter])
    else:
        full_container.append(temp_container)
        temp_container = []
        last_row = row
        temp_container.append(denoised_imgs_npy[counter])
    
    counter += 1

C:/Users/miker/Downloads/M4Raw_multicoil_test/multicoil_test\2022101101_T101.h5
C:/Users/miker/Downloads/M4Raw_multicoil_test/multicoil_test\2022101101_T101.h5
C:/Users/miker/Downloads/M4Raw_multicoil_test/multicoil_test\2022101101_T101.h5
C:/Users/miker/Downloads/M4Raw_multicoil_test/multicoil_test\2022101101_T101.h5
C:/Users/miker/Downloads/M4Raw_multicoil_test/multicoil_test\2022101101_T101.h5
C:/Users/miker/Downloads/M4Raw_multicoil_test/multicoil_test\2022101101_T101.h5
C:/Users/miker/Downloads/M4Raw_multicoil_test/multicoil_test\2022101101_T101.h5
C:/Users/miker/Downloads/M4Raw_multicoil_test/multicoil_test\2022101101_T101.h5
C:/Users/miker/Downloads/M4Raw_multicoil_test/multicoil_test\2022101101_T101.h5
C:/Users/miker/Downloads/M4Raw_multicoil_test/multicoil_test\2022101101_T101.h5
C:/Users/miker/Downloads/M4Raw_multicoil_test/multicoil_test\2022101101_T101.h5
C:/Users/miker/Downloads/M4Raw_multicoil_test/multicoil_test\2022101101_T101.h5
C:/Users/miker/Downloads/M4Raw_multicoil